# Supervised Classification

## Gender identification
Names ending in a, e and i are likely to be female, while names ending in k, o, r, s and t are likely to be male.

In [1]:
## Feature extractor
def gender_features(word):
    return {'last_letter':word[-1]}
gender_features('Shrek')

{'last_letter': 'k'}

In [2]:
from nltk.corpus import names
labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
                [(name,'female') for name in names.words('female.txt')])
import random
random.shuffle(labeled_names)

In [3]:
import nltk
featuresets = [(gender_features(n),gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:],featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(classifier.classify(gender_features('Neo')))
print(classifier.classify(gender_features('Trinity')))

male
female


In [4]:
nltk.classify.accuracy(classifier,test_set)

0.776

In [5]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     35.7 : 1.0
             last_letter = 'k'              male : female =     30.9 : 1.0
             last_letter = 'f'              male : female =     16.7 : 1.0
             last_letter = 'p'              male : female =     11.2 : 1.0
             last_letter = 'v'              male : female =     11.2 : 1.0


In [7]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features,labeled_names[:500])

## Choosing the right features

In [10]:
def gender_features2(name):
    features = {}
    features['first_letter'] = name[0].lower()
    features['last_letter'] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features['count({})'.format(letter)] = name.lower().count(letter)
        features['has({})'.format(letter)] = (letter in name.lower())
    return features
print(gender_features2('Jonh'))

{'first_letter': 'j', 'last_letter': 'h', 'count(a)': 0, 'has(a)': False, 'count(b)': 0, 'has(b)': False, 'count(c)': 0, 'has(c)': False, 'count(d)': 0, 'has(d)': False, 'count(e)': 0, 'has(e)': False, 'count(f)': 0, 'has(f)': False, 'count(g)': 0, 'has(g)': False, 'count(h)': 1, 'has(h)': True, 'count(i)': 0, 'has(i)': False, 'count(j)': 1, 'has(j)': True, 'count(k)': 0, 'has(k)': False, 'count(l)': 0, 'has(l)': False, 'count(m)': 0, 'has(m)': False, 'count(n)': 1, 'has(n)': True, 'count(o)': 1, 'has(o)': True, 'count(p)': 0, 'has(p)': False, 'count(q)': 0, 'has(q)': False, 'count(r)': 0, 'has(r)': False, 'count(s)': 0, 'has(s)': False, 'count(t)': 0, 'has(t)': False, 'count(u)': 0, 'has(u)': False, 'count(v)': 0, 'has(v)': False, 'count(w)': 0, 'has(w)': False, 'count(x)': 0, 'has(x)': False, 'count(y)': 0, 'has(y)': False, 'count(z)': 0, 'has(z)': False}


In [11]:
from nltk.classify import apply_features
train_set = apply_features(gender_features2, labeled_names[500:])
test_set = apply_features(gender_features2,labeled_names[:500])
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier,test_set))

0.784


In [14]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

train_set = apply_features(gender_features,train_names)
devtest_set = apply_features(gender_features,devtest_names)
test_set = apply_features(gender_features,test_names)
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier,devtest_set))

0.753


Examing errors when predicting devtest set to determine what additional features need to be used

In [16]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag,guess,name))

In [18]:
for (tag,guess,name) in sorted(errors):
    print('correct={:<8} guess={:<8} name={:<30}'.format(tag,guess,name))

correct=female   guess=male     name=Abagael                       
correct=female   guess=male     name=Adriaens                      
correct=female   guess=male     name=Adrien                        
correct=female   guess=male     name=Alison                        
correct=female   guess=male     name=Allis                         
correct=female   guess=male     name=Anabel                        
correct=female   guess=male     name=Arleen                        
correct=female   guess=male     name=Bell                          
correct=female   guess=male     name=Beret                         
correct=female   guess=male     name=Berget                        
correct=female   guess=male     name=Betteann                      
correct=female   guess=male     name=Bev                           
correct=female   guess=male     name=Bidget                        
correct=female   guess=male     name=Carilyn                       
correct=female   guess=male     name=Carlyn     

Looking through this list of errors makes it clear that some suffixes that are more than one letter can be indicative of name genders. For example, names ending in yn appear to be predominantly female, despite the fact that names ending in n tend to be male; and names ending in ch are usually male, even though names that end in h tend to be female. We therefore adjust our feature extractor to include features for two-letter suffixes

In [20]:
def gender_features3(word):
    return {'suffix1':word[-1:],
            'suffix2':word[-2:]}

In [21]:
train_set = apply_features(gender_features3,train_names)
devtest_set = apply_features(gender_features3,devtest_names)
test_set = apply_features(gender_features3,test_names)
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier,devtest_set))

0.785


## Document Classification

In [1]:
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)),category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]
import random
random.shuffle(documents)

In [9]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

print(document_features(movie_reviews.words('pos/cv957_8737.txt')))

{'contains(plot)': True, 'contains(:)': True, 'contains(two)': True, 'contains(teen)': False, 'contains(couples)': False, 'contains(go)': False, 'contains(to)': True, 'contains(a)': True, 'contains(church)': False, 'contains(party)': False, 'contains(,)': True, 'contains(drink)': False, 'contains(and)': True, 'contains(then)': True, 'contains(drive)': False, 'contains(.)': True, 'contains(they)': True, 'contains(get)': True, 'contains(into)': True, 'contains(an)': True, 'contains(accident)': False, 'contains(one)': True, 'contains(of)': True, 'contains(the)': True, 'contains(guys)': False, 'contains(dies)': False, 'contains(but)': True, 'contains(his)': True, 'contains(girlfriend)': True, 'contains(continues)': False, 'contains(see)': False, 'contains(him)': True, 'contains(in)': True, 'contains(her)': False, 'contains(life)': False, 'contains(has)': True, 'contains(nightmares)': False, 'contains(what)': True, "contains(')": True, 'contains(s)': True, 'contains(deal)': False, 'contains

In [10]:
featuresets = [(document_features(d),c) for (d,c) in documents]
train_set, test_set = featuresets[100:],featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier,test_set))

0.8


In [11]:
classifier.show_most_informative_features(5)

Most Informative Features
        contains(turkey) = True              neg : pos    =      7.9 : 1.0
        contains(sexist) = True              neg : pos    =      7.7 : 1.0
       contains(stellan) = True              pos : neg    =      7.6 : 1.0
 contains(unimaginative) = True              neg : pos    =      7.1 : 1.0
    contains(schumacher) = True              neg : pos    =      7.1 : 1.0


## Part-of-Speech Tagging

In [13]:
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
print(common_suffixes)

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


In [22]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endwith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n),g) for (n,g) in tagged_words]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:],featuresets[:size]
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier,test_set)

0.6270512182993535

In [23]:
classifier.classify(pos_features('cats'))

'NNS'

One nice feature of decision tree models is that they are often fairly easy to interpret — we can even instruct NLTK to print them out as pseudocode

In [24]:
print(classifier.pseudocode(depth=4))

if endwith(the) == False: 
  if endwith(,) == False: 
    if endwith(s) == False: 
      if endwith(.) == False: return '.'
      if endwith(.) == True: return '.'
    if endwith(s) == True: 
      if endwith(is) == False: return 'PP$'
      if endwith(is) == True: return 'BEZ'
  if endwith(,) == True: return ','
if endwith(the) == True: return 'AT'



## Exploiting Context

In [26]:
def pos_features(sentence,i):
    features = {"suffix(1)":sentence[i][-1:],
               "suffix(2)":sentence[i][-2:],
               "suffix(3)":sentence[i][-3:]}
    if i==0:
        features['prev-word'] = '<START>'
    else:
        features['prev_word'] = sentence[i-1]
    return features
print(pos_features(brown.sents()[0],8))

{'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev_word': 'an'}


In [28]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word,tag) in enumerate(tagged_sent):
        featuresets.append((pos_features(untagged_sent,i),tag))
        
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:],featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

nltk.classify.accuracy(classifier,test_set)

0.7895574341123819

## Sequence Classification

In [32]:
def pos_features(sentence,i,history):
    features = {"suffix(1)":sentence[i][-1:],
               "suffix(2)":sentence[i][-2:],
               "suffix(3)":sentence[i][-3:]}
    if i==0:
        features['prev-word'] = '<START>'
        features['prev-tag'] = '<START>'
    else:
        features['prev-word'] = sentence[i-1]
        features['prev-tag'] = history[i-1]
    return features

class ConsecutivePosTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
        
    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [33]:
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:],tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
print(tagger.evaluate(test_sents))

0.7980528511821975


# Further Examples of Supervised Classification
## Sentence Segmentation

In [35]:
sents = nltk.corpus.treebank_raw.sents()
tokens = []
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset-1)

In [44]:
def punct_features(tokens,i):
    return {'next-word-capitalized':tokens[i+1][0].isupper(),
           'prev-word':tokens[i-1].lower(),
           'punct':tokens[i],
           'prev-word-is-one-char':len(tokens[i-1])==1}

featuresets = [(punct_features(tokens,i), (i in boundaries))
              for i in range(1,len(tokens)-1)
              if tokens[i] in '.?!']

size = int(len(featuresets)*0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier,test_set)

0.936026936026936

Use the above classifier to segement sentences

In [ ]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if word in '.?!' and classifier.classify(punct_features(words,i))==True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents

## Identifying Dialogue Act Types
Greetings, questions, answers, assertions, and clarifications can all be thought of as types of speech-based actions. Recognizing the dialogue acts underlying the utterances in a dialogue can be an important first step in understanding the conversation.

In [48]:
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

In [52]:
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

featuresets = [(dialogue_act_features(post.text), post.get('class'))
              for post in posts]
size = int(len(featuresets)*0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier,test_set)

0.668

## Recognizing Textual Entailment
Recognizing textual entailment (RTE) is the task of determining whether a given piece of text T entails another text called the "hypothesis".
It should be emphasized that the relationship between text and hypothesis is not intended to be logical entailment, but rather whether a human would conclude that the text provides reasonable evidence for taking the hypothesis to be true.

In [53]:
def rte_features(rtepair):
    extractor = nltk.RTEFeatureExtractor(rtepair)
    features = {}
    features['word_overlap']=len(extractor.overlap('word'))
    features['word_hyp_extra']=len(extractor.hyp_extra('word'))
    features['ne_overlap']=len(extractor.overlap('ne'))
    features['ne_hyp_extra'] = len(extractor.hyp_extra('ne'))
    return features

In [59]:
rtepair = nltk.corpus.rte.pairs(['rte3_dev.xml'])[33]
extractor = nltk.RTEFeatureExtractor(rtepair)
print(extractor.text_words)
print()
print(extractor.hyp_words)
print()
print(extractor.overlap('word'))
print()
print(extractor.overlap('ne'))
print()
print(extractor.hyp_extra('word'))

{'together', 'Iran', 'central', 'Russia', 'republics', 'fight', 'terrorism.', 'Co', 'Davudi', 'that', 'Organisation', 'fledgling', 'China', 'binds', 'four', 'former', 'operation', 'Shanghai', 'was', 'Soviet', 'meeting', 'Asia', 'SCO', 'Parviz', 'representing', 'at', 'association'}

{'member', 'SCO.', 'China'}

set()

{'China'}

{'member'}


# Decision Tree
## Entropy and Information Gain

In [66]:
import math
def entropy(labels):
    freqdist = nltk.FreqDist(labels)
    print(freqdist)
    probs = [freqdist.freq(l) for l in freqdist]
    print(probs)
    return -sum(p*math.log(p,2) for p in probs)

In [62]:
print(entropy(['male', 'male', 'male', 'male'])) 
print(entropy(['male', 'female', 'male', 'male']))
print(entropy(['female', 'male', 'female', 'male']))
print(entropy(['female', 'female', 'male', 'female']))
print(entropy(['female', 'female', 'female', 'female'])) 

-0.0
0.8112781244591328
1.0
0.8112781244591328
-0.0


### Useful qualities of Decision Trees
- Easy to interpret
- Decision trees are especially well suited to cases where many hierarchical categorical distinctions can be made

### Disadvantages
- Easily overfit the training data
- They force features to be checked in a specific order, even when features may act relatively independently of one another.

# Naive Bayes Classifiers
In naive Bayes classifiers, every feature gets a say in determining which label should be assigned to a given input value. To choose a label for an input value, the naive Bayes classifier begins by calculating the prior probability of each label, which is determined by checking frequency of each label in the training set. The contribution from each feature is then combined with this prior probability, to arrive at a likelihood estimate for each label. The label whose likelihood estimate is the highest is then assigned to the input value.

# Maximum Entropy Classifiers
The Maximum Entropy classifier uses a model that is very similar to the model employed by the naive Bayes classifier. But rather than using probabilities to set the model's parameters, it uses search techniques to find a set of parameters that will maximize the performance of the classifier. In particular, it looks for the set of parameters that maximizes the total likelihood of the training corpus

An important difference between the naive Bayes classifier and the Maximum Entropy classifier concerns the type of questions they can be used to answer. The naive Bayes classifier is an example of a generative classifier, which builds a model that predicts P(input, label), the joint probability of a (input, label) pair. As a result, generative models can be used to answer the following questions:

1. What is the most likely label for a given input?
2. How likely is a given label for a given input?
3. What is the most likely input value?
4. How likely is a given input value?
5. How likely is a given input value with a given label?
6. What is the most likely label for an input that might have one of two values (but we don't know which)?

The Maximum Entropy classifier, on the other hand, is an example of a conditional classifier. Conditional classifiers build models that predict P(label|input) — the probability of a label given the input value. Thus, conditional models can still be used to answer questions 1 and 2. However, conditional models can not be used to answer the remaining questions 3-6.

In general, generative models are strictly more powerful than conditional models, since we can calculate the conditional probability P(label|input) from the joint probability P(input, label), but not vice versa. However, this additional power comes at a price. Because the model is more powerful, it has more "free parameters" which need to be learned. However, the size of the training set is fixed. Thus, when using a more powerful model, we end up with less data that can be used to train each parameter's value, making it harder to find the best parameter values. As a result, a generative model may not do as good a job at answering questions 1 and 2 as a conditional model, since the conditional model can focus its efforts on those two questions. However, if we do need answers to questions like 3-6, then we have no choice but to use a generative model.

# What do models tell us
It's important to understand what we can learn about language from an automatically constructed model. One important consideration when dealing with models of language is the distinction between descriptive models and explanatory models. Descriptive models capture patterns in the data but they don't provide any information about why the data contains those patterns. For example, as we saw in 3.1, the synonyms absolutely and definitely are not interchangeable: we say absolutely adore not definitely adore, and definitely prefer not absolutely prefer. In contrast, explanatory models attempt to capture properties and relationships that cause the linguistic patterns. For example, we might introduce the abstract concept of "polar verb", as one that has an extreme meaning, and categorize some verb like adore and detest as polar. Our explanatory model would contain the constraint that absolutely can only combine with polar verbs, and definitely can only combine with non-polar verbs. In summary, descriptive models provide information about correlations in the data, while explanatory models go further to postulate causal relationships.